In [1]:
# import gensim.downloader as api

# info = api.info()  # show info about available models/datasets
# #word2vec-google-news-300 , 
# model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
# model.wv("cat")

In [2]:
#len(model.wv["cat"])

In [3]:
import pandas as pd
from gensim.models import Word2Vec
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk import tokenize
from nltk import word_tokenize
from gensim.models import Phrases

In [4]:
emscad = pd.read_csv("/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/sentences/sentences_emscad.csv")
emscad = emscad[['sentence', 'sentence_lemmatized', 'sentence_no_stopwords']]

emscad = pd.concat([emscad['sentence'],emscad['sentence_lemmatized'],emscad['sentence_no_stopwords'] ]).reset_index()
emscad = emscad[[0]]
df = emscad.rename(columns={0:'job_description'})
emscad_sentences = df['job_description'].tolist()
emscad_sentences

['food52 a fast growing james beard award winning online food community and crowd sourced and curated recipe hub is currently interviewing full and part time unpaid interns to work in a small team of editors executives and developers in its new york city headquarters',
 'reproducing and or repackaging existing food52 content for a number of partner sites such as huffington post yahoo buzzfeed and more in their various content management systems',
 'researching blogs and websites for the provisions by food52 affiliate program',
 'assisting in day to day affiliate program support such as screening affiliates and assisting in any affiliate inquiries',
 'supporting with pr  events when needed',
 'helping with office administrative work such as filing mailing and preparing for meetings',
 'working with developers to document bugs and suggest improvements to the site',
 'supporting the marketing and executive staff',
 'experience with content management systems a major plus any blogging coun

In [5]:
reed = pd.read_csv('/Users/ivowings/Sync/Thesis/Datasources/Job vacancies/reed_uk.csv')
reed['job_requirements'] = reed['job_requirements'].fillna('')
reed['job_description'] = reed['job_description'] + ' ' + reed['job_requirements']
df2 = reed[['job_description']]
df = pd.concat([df2])
df

,job_description
0,Apply now New opportunity not to be missed! H...
1,Apply now This role requires an experienced s...
2,Apply now Job Title: Warehouse personLocation...
3,Apply now Castles Solicitors are looking for ...
4,Apply now International Pharmaceutical Manufa...
...,...
49995,Apply now Scientist - Cell Based Assays - Neu...
49996,Apply now Our client is looking for high achi...
49997,Apply on employer's website Who youâ€™ll be w...
49998,"Apply now UNDERSTANDING, SERVICE, TRUST3 Word..."


In [6]:
tqdm.pandas()
sentences = []

#Breaking the job postings into sentences

for vacancy in tqdm(df['job_description']):
    sentences_storage = tokenize.sent_tokenize(vacancy)
    for sentence in sentences_storage:
        if sentence != '':
            sentences.append(sentence)
    
    

100%|██████████| 50000/50000 [00:24<00:00, 2043.11it/s]


In [7]:
from nltk.tokenize import RegexpTokenizer

#Combining the reed set and the emscad sentences

sentences = sentences + emscad_sentences

#Removing the punctiation from the sentences
#Tokeninzing the sentences and appending each sentence to its own list
tokenizer = RegexpTokenizer(r'\w+')
sentences_cleaned = []
for sentence in tqdm(sentences):
    tokens = tokenizer.tokenize(str(sentence))
    sentences_cleaned.append(tokens)

100%|██████████| 1371471/1371471 [00:15<00:00, 88946.97it/s] 


In [8]:
#Lowercasing all the text
sentences_cleaned_final = []
for sentence in tqdm(sentences_cleaned):
    sublist = []
    for token in sentence:
        sublist.append(token.lower())
    sentences_cleaned_final.append(sublist)
        #sentences_cleaned_final.append(token.lower())
        

100%|██████████| 1371471/1371471 [00:11<00:00, 120641.32it/s]


In [9]:
sentences_cleaned_final

[['apply', 'now', 'new', 'opportunity', 'not', 'to', 'be', 'missed'],
 ['haulfryn',
  'group',
  'has',
  'a',
  'fantastic',
  'opportunity',
  'for',
  'a',
  'commis',
  'chef',
  'to',
  'join',
  'a',
  'growing',
  'team',
  'at',
  'our',
  'friendly',
  'holiday',
  'resort',
  'finlake',
  'holiday',
  'resort'],
 ['in',
  'return',
  'we',
  'offer',
  'our',
  'commis',
  'chef',
  'a',
  'competitive',
  'salary',
  'depending',
  'on',
  'experience',
  'plus',
  'excellent',
  'benefits'],
 ['working',
  'at',
  'our',
  'prestigious',
  'and',
  'exclusive',
  'holiday',
  'park',
  'with',
  'owners',
  'who',
  'expect',
  'nothing',
  'but',
  'the',
  'best',
  'we',
  'need',
  'an',
  'exceptional',
  'person',
  'to',
  'join',
  'our',
  'kitchen',
  'team'],
 ['weâ',
  're',
  'looking',
  'for',
  'an',
  'organised',
  'enthusiastic',
  'and',
  'standards',
  'oriented',
  'food',
  'professional'],
 ['our',
  'ideal',
  'commis',
  'chef',
  'will',
  'have'

In [10]:
%%time
#sg = 1 for skip-gram 0 CBOW.
model = Word2Vec(sentences=sentences_cleaned_final, size=100, min_count=1, workers=4, sg=0)
model.save("/Users/ivowings/Sync/Thesis/Code/Word2vec models/word2vec_s100_CBOW.model")

CPU times: user 6min 2s, sys: 3.75 s, total: 6min 6s
Wall time: 2min 14s


In [11]:
%%time
#sg = 1 for skip-gram 0 CBOW.
model = Word2Vec(sentences=sentences_cleaned_final, size=200, min_count=1, workers=4, sg=0)
model.save("/Users/ivowings/Sync/Thesis/Code/Word2vec models/word2vec_s200_CBOW.model")

CPU times: user 7min 12s, sys: 8.1 s, total: 7min 20s
Wall time: 3min 40s


In [12]:
%%time
#sg = 1 for skip-gram 0 CBOW.
model = Word2Vec(sentences=sentences_cleaned_final, size=100, min_count=1, workers=4, sg=1)
model.save("/Users/ivowings/Sync/Thesis/Code/Word2vec models/word2vec_s100_SKIP.model")

CPU times: user 17min 52s, sys: 8.92 s, total: 18min 1s
Wall time: 6min 6s


In [13]:
%%time
#sg = 1 for skip-gram 0 CBOW.
model = Word2Vec(sentences=sentences_cleaned_final, size=200, min_count=1, workers=4, sg=1)
model.save("/Users/ivowings/Sync/Thesis/Code/Word2vec models/word2vec_s200_SKIP.model")

CPU times: user 21min 28s, sys: 16.1 s, total: 21min 45s
Wall time: 7min 53s


In [14]:
%%time
model = Word2Vec(sentences=sentences_cleaned_final)
model.save("/Users/ivowings/Sync/Thesis/Code/Word2vec models/word2vec_default.model")

CPU times: user 5min 17s, sys: 3.61 s, total: 5min 20s
Wall time: 2min 37s
